In [27]:
import pandas as pd

Read in the 1920_2024_geopy and 

In [28]:
df = pd.read_json("data/silver/1920_2024_geopy.json")
df[df.Country_location.apply(lambda x: isinstance(x,str))].to_csv("data/bad_data/unknown_country_location.csv")
df_ = df[df.Country_location.apply(lambda x: isinstance(x,dict))]
df_[:1]

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location
0,1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,None,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0.0,Crashed into approach lights while attempting ...,35,27,British European Airways,"{'place_id': 372303099, 'licence': 'Data © Ope..."


In [29]:
df_country_codes = pd.read_json("data/country_codes.json")
df_country_codes[df_country_codes.Country_location.apply(lambda x: isinstance(x,str))].to_csv("data/bad_data/unknown_country_codes.csv")
df_country_codes_ = df_country_codes[df_country_codes.Country_location.apply(lambda x: isinstance(x,dict))]
df_country_codes_ = df_country_codes_[df_country_codes_.name.apply(lambda x: x not in ['Palestine, State of', 'Holy See'])]
df_country_codes[:4]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,Country_location
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN,"{'place_id': 220100631, 'licence': 'Data © Ope..."
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,None,150.0,154.0,NaN,"{'place_id': 180167715, 'licence': 'Data © Ope..."
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN,"{'place_id': 83081284, 'licence': 'Data © Open..."
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 77146891, 'licence': 'Data © Open..."


In [30]:
len(df_country_codes),len(df_)

(249, 4918)

In [31]:

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def unpack_locations(x,a=-2):
    "Check if is numeric"
    return ",".join([i for i in x['display_name'].split(",") if i.strip().isnumeric()==False][a:]).strip()

def with_conditions(x):
    if has_numbers(unpack_locations(x,a=-2)):
        return unpack_locations(x,a=-1)
    return unpack_locations(x,a=-2)

def get_country_from_dict(df):
    df["Country_Geopy"] = df.Country_location.apply(with_conditions)
    df = df.set_index("Country_Geopy")
    return df

def check_and_compare(dfContryCode,seriesIndex):
    return dfContryCode.loc[seriesIndex]["alpha-3"]


def get_code(country_code,x):
    try: 
        try:
            return  check_and_compare(country_code, x)
        except:
            return check_and_compare(country_code, x.split(",")[-1].strip())
    except:
        return float("nan")

def get_single_data_frame(x):
    return x 


In [32]:
country_code = get_country_from_dict(df_country_codes_)
data_df = get_country_from_dict(df_)

/var/folders/sr/32z6khr97xs2g11w52pbkzww0000gn/T/ipykernel_75293/1916793209.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Country_Geopy"] = df.Country_location.apply(with_conditions)


In [33]:
data_df["Code"]=data_df.index.map(lambda x: get_code(country_code,x))
data_df

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location,Code
Country_Geopy,,,,,,,,,,,,,,,,,,,
"Northern Ireland, United Kingdom",1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,None,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0.0,Crashed into approach lights while attempting ...,35,27,British European Airways,"{'place_id': 372303099, 'licence': 'Data © Ope...",GBR
"Washington, United States",1950,1953-01-07,2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0.0,The plane struck a tree on a mountain ridge at...,7,7,Flying Tiger Line,"{'place_id': 383391892, 'licence': 'Data © Ope...",USA
"Idaho, United States",1950,1953-01-07,0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0.0,The plane crashed into mountains while en rout...,40,40,Associated Air Transport,"{'place_id': 315821592, 'licence': 'Data © Ope...",USA
Italy,1950,1953-01-15,0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,None,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,None,"{'total': 26, 'passengers': 16, 'crew': 10}","{'total': 26, 'passengers': 16, 'crew': 10}",0.0,Both aircraft crashed after a midair collision...,26,26,Military,"{'place_id': 100666283, 'licence': 'Data © Ope...",ITA
"Sardinia, Italy",1950,1953-01-26,1144,"Sinnai, Sardinia, Italy",Linee Aeree Italiane,None,Cagliari - Rome,Douglas DC-3,I-LAIL,4308,"{'total': 19, 'passengers': 15, 'crew': 4}","{'total': 19, 'passengers': 15, 'crew': 4}",0.0,Crashed 10 miles east of Cagliari in the Sinna...,19,19,Linee Aeree Italiane,"{'place_id': 78523394, 'licence': 'Data © Open...",ITA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Siberian Federal District, Russia",1970,1976-12-17,1940,"Ust-Kut, Russia",Aeroflot,None,Ust-Kut - Kirensk,Yakovlev 40,CCCP-88208,9631349,"{'total': 7, 'passengers': 4, 'crew': 3}","{'total': 7, 'passengers': 4, 'crew': 3}",0.0,The cargo plane crashed into trees on takeoff....,7,7,Aeroflot,"{'place_id': 224774650, 'licence': 'Data © Ope...",RUS
"Kyiv, Ukraine",1970,1976-12-17,21:30,"Kiev, USSR",Aeroflot,H-36,Chernivtsi - Kiev,Antonov AN-24RV,CCCP-46672,47309604,"{'total': 55, 'passengers': 50, 'crew': 5}","{'total': 48, 'passengers': 44, 'crew': 4}",0.0,In heavy fog the crew continued their descend...,55,48,Aeroflot,"{'place_id': 202588741, 'licence': 'Data © Ope...",UKR
Russia,1970,1976-12-18,1656,"Nnear Yuzhno-Sakhalinsk, Russia",Aeroflot,None,Petropavlovsk-Kamchatsky - Yuzhno-Sakhalinsk,Ilyushin IL-14M,CCCP-61752,147001247,"{'total': 8, 'passengers': 3, 'crew': 5}","{'total': 8, 'passengers': 3, 'crew': 5}",0.0,The survey fllight deviated from the approach ...,8,8,Aeroflot,"{'place_id': 173174605, 'licence': 'Data © Ope...",RUS


In [34]:
df = data_df

In [35]:
df 

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location,Code
Country_Geopy,,,,,,,,,,,,,,,,,,,
"Northern Ireland, United Kingdom",1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,None,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0.0,Crashed into approach lights while attempting ...,35,27,British European Airways,"{'place_id': 372303099, 'licence': 'Data © Ope...",GBR
"Washington, United States",1950,1953-01-07,2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0.0,The plane struck a tree on a mountain ridge at...,7,7,Flying Tiger Line,"{'place_id': 383391892, 'licence': 'Data © Ope...",USA
"Idaho, United States",1950,1953-01-07,0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0.0,The plane crashed into mountains while en rout...,40,40,Associated Air Transport,"{'place_id': 315821592, 'licence': 'Data © Ope...",USA
Italy,1950,1953-01-15,0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,None,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,None,"{'total': 26, 'passengers': 16, 'crew': 10}","{'total': 26, 'passengers': 16, 'crew': 10}",0.0,Both aircraft crashed after a midair collision...,26,26,Military,"{'place_id': 100666283, 'licence': 'Data © Ope...",ITA
"Sardinia, Italy",1950,1953-01-26,1144,"Sinnai, Sardinia, Italy",Linee Aeree Italiane,None,Cagliari - Rome,Douglas DC-3,I-LAIL,4308,"{'total': 19, 'passengers': 15, 'crew': 4}","{'total': 19, 'passengers': 15, 'crew': 4}",0.0,Crashed 10 miles east of Cagliari in the Sinna...,19,19,Linee Aeree Italiane,"{'place_id': 78523394, 'licence': 'Data © Open...",ITA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Siberian Federal District, Russia",1970,1976-12-17,1940,"Ust-Kut, Russia",Aeroflot,None,Ust-Kut - Kirensk,Yakovlev 40,CCCP-88208,9631349,"{'total': 7, 'passengers': 4, 'crew': 3}","{'total': 7, 'passengers': 4, 'crew': 3}",0.0,The cargo plane crashed into trees on takeoff....,7,7,Aeroflot,"{'place_id': 224774650, 'licence': 'Data © Ope...",RUS
"Kyiv, Ukraine",1970,1976-12-17,21:30,"Kiev, USSR",Aeroflot,H-36,Chernivtsi - Kiev,Antonov AN-24RV,CCCP-46672,47309604,"{'total': 55, 'passengers': 50, 'crew': 5}","{'total': 48, 'passengers': 44, 'crew': 4}",0.0,In heavy fog the crew continued their descend...,55,48,Aeroflot,"{'place_id': 202588741, 'licence': 'Data © Ope...",UKR
Russia,1970,1976-12-18,1656,"Nnear Yuzhno-Sakhalinsk, Russia",Aeroflot,None,Petropavlovsk-Kamchatsky - Yuzhno-Sakhalinsk,Ilyushin IL-14M,CCCP-61752,147001247,"{'total': 8, 'passengers': 3, 'crew': 5}","{'total': 8, 'passengers': 3, 'crew': 5}",0.0,The survey fllight deviated from the approach ...,8,8,Aeroflot,"{'place_id': 173174605, 'licence': 'Data © Ope...",RUS


In [36]:
df = data_df

In [37]:
df["Country_Geopy"] = df.index
df["index"] = range(len(df))

In [38]:
df = df.set_index("index")

In [39]:
df.to_csv("data/ready_for_plotting/data_plot_map.csv",index=False)

In [40]:
data_df.to_json("data/ready_for_plotting/data_plot_map.json",index_call=False)

TypeError: NDFrame.to_json() got an unexpected keyword argument 'index_call'

In [ ]:
# country_code.loc["Gangwon State, South Korea"]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,Country_location
Country_Geopy,,,,,,,,,,,,
"Gangwon State, South Korea",Korea (Democratic People's Republic of),KP,PRK,408,ISO 3166-2:KP,Asia,Eastern Asia,None,142.0,30.0,NaN,"{'place_id': 227836027, 'licence': 'Data © Ope..."
"Gangwon State, South Korea","Korea, Republic of",KR,KOR,410,ISO 3166-2:KR,Asia,Eastern Asia,None,142.0,30.0,NaN,"{'place_id': 227743938, 'licence': 'Data © Ope..."


In [ ]:
for i in country_code.index:
    print(i)

Afghanistan
Åland Islands, Finland
Albania
Algeria
American Samoa, United States
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda
Argentina
Armenia
Aruba, Netherlands
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia
Caribbean Netherlands, Netherlands
Bosnia and Herzegovina
Botswana
Bouvet Island, Norway
Brazil
British Indian Ocean Territory
Brunei
Bulgaria
Burkina Faso
Burundi
Cape Verde
Cambodia
Cameroon
Canada
Cayman Islands
Central African Republic
Chad
Chile
China
Christmas Island, Australia
Cocos (Keeling) Islands, Australia
Colombia
Comoros
Democratic Republic of the Congo
Democratic Republic of the Congo
Cook Islands
Costa Rica
Côte d'Ivoire
Croatia
Cuba
Curacao, Netherlands
Cyprus
Czechia
Denmark
Djibouti
Dominican Republic
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Falkland Islands
Faroe Islands
Fiji
Finland
France
French Guiana, France
French Polynes

In [ ]:
print(data_df[data_df.Code.apply(lambda x: isinstance(x,float))].Location.to_string())

Country_Geopy
North Korea                                                           Near Kanggye, North Korea
Pacific Ocean                                                          China Sea, Pacific Ocean
North Pacific Ocean                                                         North Pacific Ocean
Persian Gulf                                                              Bahrain, Persian Gulf
South Korea                                                           Near Hunghae, South Korea
New Taipei, Taiwan                                                                Wanli, Taiwan
Indian Ocean                                            Off Ceylon, Cocos Islands, Indian Ocean
North Pacific Ocean                                                         North Pacific Ocean
Atlantic Ocean                                                        Caribbean, Atlantic Ocean
Baltic Sea                                                                           Baltic Sea
North Sea                 

In [ ]:
check_and_compare(country_code)

In [ ]:
df_[df_.Country_Geopy.apply(lambda x: has_numbers(x))]

In [ ]:
country_code = get_country_from_dict(df_country_codes_)
country_code[:3]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,Country_location
Country_Geopy,,,,,,,,,,,,
Afghanistan,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN,"{'place_id': 220100631, 'licence': 'Data © Ope..."
"Åland Islands, Finland",Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,None,150.0,154.0,NaN,"{'place_id': 180167715, 'licence': 'Data © Ope..."
Albania,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN,"{'place_id': 83081284, 'licence': 'Data © Open..."


In [ ]:
country_code.loc["United States"]

name                                                 United States of America
alpha-2                                                                    US
alpha-3                                                                   USA
country-code                                                              840
iso_3166-2                                                      ISO 3166-2:US
region                                                               Americas
sub-region                                                   Northern America
intermediate-region                                                      None
region-code                                                              19.0
sub-region-code                                                          21.0
intermediate-region-code                                                  NaN
Country_location            {'place_id': 77249893, 'licence': 'Data © Open...
Name: United States, dtype: object

In [ ]:
data_df = get_country_from_dict(df_)
data_df[:3]

/var/folders/sr/32z6khr97xs2g11w52pbkzww0000gn/T/ipykernel_66474/3156633071.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Country_Geopy"] = df.Country_location.apply(with_conditions)


,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location
Country_Geopy,,,,,,,,,,,,,,,,,,
"Northern Ireland, United Kingdom",1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,None,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0.0,Crashed into approach lights while attempting ...,35,27,British European Airways,"{'place_id': 372303099, 'licence': 'Data © Ope..."
"Washington, United States",1950,1953-01-07,2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0.0,The plane struck a tree on a mountain ridge at...,7,7,Flying Tiger Line,"{'place_id': 383391892, 'licence': 'Data © Ope..."
"Idaho, United States",1950,1953-01-07,0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0.0,The plane crashed into mountains while en rout...,40,40,Associated Air Transport,"{'place_id': 315821592, 'licence': 'Data © Ope..."


In [ ]:
# 'This country match >>> Holy See >>> Texas, United States', red flag here Holy See is being tide to Texas so drop 
# This list comprehension checks if the name is in the index if it's not in there we list it to check sanity 
def check_index_equal_to_series(sr):
    [print(f"This country match >>> {sr[sr == i].index[0]} >>> {i} ") for i in sr if i not in sr[sr == i].index[0]]
check_index_equal_to_series(country_code["name"])

This country match >>> Bolivia >>> Bolivia (Plurinational State of) 
This country match >>> Caribbean Netherlands, Netherlands >>> Bonaire, Sint Eustatius and Saba 
This country match >>> Brunei >>> Brunei Darussalam 
This country match >>> Cape Verde >>> Cabo Verde 
This country match >>> Democratic Republic of the Congo >>> Congo, Democratic Republic of the 
This country match >>> Curacao, Netherlands >>> Curaçao 
This country match >>> Falkland Islands >>> Falkland Islands (Malvinas) 
This country match >>> French Southern and Antarctic Lands, France >>> French Southern Territories 
This country match >>> Iran >>> Iran (Islamic Republic of) 
This country match >>> Gangwon State, South Korea >>> Korea (Democratic People's Republic of) 
This country match >>> Gangwon State, South Korea >>> Korea, Republic of 
This country match >>> Laos >>> Lao People's Democratic Republic 
This country match >>> Macau, China >>> Macao 
This country match >>> Federated States of Micronesia >>> Microne

In [ ]:
check_index_equal_to_series(data_df["Location"])

This country match >>> Northern Ireland, United Kingdom >>> Belfast, Northern Ireland 
This country match >>> Washington, United States >>> Issaquah, Washington 
This country match >>> Idaho, United States >>> Fish Haven, Idaho 
This country match >>> Italy >>> 25 nm off Agrigento, Italy 
This country match >>> Sardinia, Italy >>> Sinnai, Sardinia, Italy 
This country match >>> Newfoundland and Labrador, Canada >>> Off Newfoundland, Canada 
This country match >>> Lai Châu Province, Vietnam >>> Lai Chau, Vietnam 
This country match >>> Metropolitan France, France >>> Bordeaux, France 
This country match >>> Alabama, United States >>> Off Mobile, Alabama 
This country match >>> Sindh, Pakistan >>> Karachi, Pakistan 
This country match >>> Connecticut, United States >>> Windsor Locks, Connecticut 
This country match >>> India >>> Kalasahar, India 
This country match >>> Vietnam >>> Off Da Nang, Vietnam 
This country match >>> Newfoundland and Labrador, Canada >>> Random Island, Newfoundla

In [ ]:
# Geopy is tagging 
data_df[:4]

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location
Country_Geopy,,,,,,,,,,,,,,,,,,
"Northern Ireland, United Kingdom",1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,None,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0.0,Crashed into approach lights while attempting ...,35,27,British European Airways,"{'place_id': 372303099, 'licence': 'Data © Ope..."
"Washington, United States",1950,1953-01-07,2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0.0,The plane struck a tree on a mountain ridge at...,7,7,Flying Tiger Line,"{'place_id': 383391892, 'licence': 'Data © Ope..."
"Idaho, United States",1950,1953-01-07,0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0.0,The plane crashed into mountains while en rout...,40,40,Associated Air Transport,"{'place_id': 315821592, 'licence': 'Data © Ope..."
Italy,1950,1953-01-15,0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,None,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,None,"{'total': 26, 'passengers': 16, 'crew': 10}","{'total': 26, 'passengers': 16, 'crew': 10}",0.0,Both aircraft crashed after a midair collision...,26,26,Military,"{'place_id': 100666283, 'licence': 'Data © Ope..."


In [ ]:
df_["Code"] = df_.Country_Geopy.apply(get_code)

/var/folders/sr/32z6khr97xs2g11w52pbkzww0000gn/T/ipykernel_64845/3812215558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["Code"] = df_.Country_Geopy.apply(get_code)


In [ ]:
df_["Code"].info()

<class 'pandas.core.series.Series'>
Index: 4918 entries, 0 to 5036
Series name: Code
Non-Null Count  Dtype 
--------------  ----- 
1532 non-null   object
dtypes: object(1)
memory usage: 205.9+ KB


In [ ]:
good_code_column = df_[df_.Code.apply(lambda x: not isinstance(x,float))]
missing_code = df_[df_.Code.apply(lambda x: isinstance(x,float))]
good_code_column[:2]

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location,Country_Geopy,Code
3,1950,1953-01-15,0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,None,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,None,"{'total': 26, 'passengers': 16, 'crew': 10}","{'total': 26, 'passengers': 16, 'crew': 10}",0.0,Both aircraft crashed after a midair collision...,26,26,Military,"{'place_id': 100666283, 'licence': 'Data © Ope...",Italy,ITA
12,1950,1953-03-14,None,"Kalasahar, India",Orient Airways,None,Delhi - Dacca,Convair CV-240-7,AP-AEG,082,"{'total': 16, 'passengers': 11, 'crew': 5}","{'total': 16, 'passengers': 11, 'crew': 5}",0.0,Crashed into a mountain while en route in seve...,16,16,Orient Airways,"{'place_id': 246884012, 'licence': 'Data © Ope...",India,IND


In [ ]:
# All the missing data from USA we have 50 different states which checks out 
len({i for i in missing_code.Country_Geopy if "United States" in i})

50

In [ ]:
missing_code.columns

Index(['Decade', 'Date', 'Time', 'Location', 'Operator', 'Flight_Num', 'Route',
       'Ac_Type', 'Registration', 'Cn_Ln', 'Aboard', 'Fatalities', 'Ground',
       'Summary', 'Aboard_Total', 'Fatalities_Total', 'Operator_vs_Military',
       'Country_location', 'Country_Geopy', 'Code'],
      dtype='object')

In [ ]:
def check_country_geopy_match(s):
    return lambda x : s in x

def get_missing_df(df, s):
    return df[df.Country_Geopy.apply(check_country_geopy_match(s))]

In [ ]:
df_country_codes_updated

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,name_geopy
Country_Geopy,,,,,,,,,,,,
Afghanistan,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN,"{'place_id': 220100631, 'licence': 'Data © Ope..."
"Åland Islands, Finland",Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,None,150.0,154.0,NaN,"{'place_id': 180167715, 'licence': 'Data © Ope..."
Albania,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN,"{'place_id': 83081284, 'licence': 'Data © Open..."
Algeria,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 77146891, 'licence': 'Data © Open..."
"American Samoa, United States",American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,None,9.0,61.0,NaN,"{'place_id': 32805879, 'licence': 'Data © Open..."
...,...,...,...,...,...,...,...,...,...,...,...,...
"Wallis and Futuna, France",Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,None,9.0,61.0,NaN,"{'place_id': 33145748, 'licence': 'Data © Open..."
"Dakhla-Oued Ed-Dahab, Morocco",Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 288513039, 'licence': 'Data © Ope..."
Yemen,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,None,142.0,145.0,NaN,"{'place_id': 69324426, 'licence': 'Data © Open..."


In [ ]:
print(get_missing_df(missing_code, "United States")[["Location","Country_Geopy","Operator"]].to_string())

                                                          Location                        Country_Geopy                                                           Operator
1                                             Issaquah, Washington            Washington, United States                                                  Flying Tiger Line
2                                                Fish Haven, Idaho                 Idaho, United States                                           Associated Air Transport
9                                              Off Mobile, Alabama               Alabama, United States                                                  National Airlines
11                                      Windsor Locks, Connecticut           Connecticut, United States                                                      Slick Airways
16                                            Alvarado, California            California, United States                                          

In [ ]:
missing_code["Country_Geop_3"] = missing_code.Country_location.apply(unpack_locations_part)

NameError: name 'unpack_locations_part' is not defined

In [ ]:
missing_code["Code_2"] = missing_code.Country_Geopy.apply(get_code)

/var/folders/sr/32z6khr97xs2g11w52pbkzww0000gn/T/ipykernel_64296/2062250936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_code["Code_2"] = missing_code.Country_Geopy.apply(get_code)


In [ ]:
missing_code

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,...,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location,Country_Geopy,Code,Country_Geop_3,Code_2
0,1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,None,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,...,0.0,Crashed into approach lights while attempting ...,35,27,British European Airways,"{'place_id': 372303099, 'licence': 'Data © Ope...","Northern Ireland, United Kingdom",NaN,United Kingdom,NaN
1,1950,1953-01-07,2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,...,0.0,The plane struck a tree on a mountain ridge at...,7,7,Flying Tiger Line,"{'place_id': 383391892, 'licence': 'Data © Ope...","Washington, United States",NaN,United States,NaN
2,1950,1953-01-07,0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,...,0.0,The plane crashed into mountains while en rout...,40,40,Associated Air Transport,"{'place_id': 315821592, 'licence': 'Data © Ope...","Idaho, United States",NaN,United States,NaN
4,1950,1953-01-26,1144,"Sinnai, Sardinia, Italy",Linee Aeree Italiane,None,Cagliari - Rome,Douglas DC-3,I-LAIL,4308,...,0.0,Crashed 10 miles east of Cagliari in the Sinna...,19,19,Linee Aeree Italiane,"{'place_id': 78523394, 'licence': 'Data © Open...","Sardinia, Italy",NaN,Italy,NaN
5,1950,1953-02-02,0530,"Off Newfoundland, Canada",Lancashire Aircraft Corporation (Skyways),None,"Lajes, Azores - Gander",Avro 685 York 1,G-AHFA,1304,...,0.0,"Last know coordinates 46' 15"" N, 46'31""W, over...",39,39,Lancashire Aircraft Corporation (Skyways),"{'place_id': 278097010, 'licence': 'Data © Ope...","Newfoundland and Labrador, Canada",NaN,Canada,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5029,1970,1976-11-23,0945,"Kozani, Greece",Olympic Airways,None,Athens - Larisa,NAMC YS-11A-500,SX-BBR,2156,...,0.0,The airliner crashed into mountainous terrain ...,50,50,Olympic Airways,"{'place_id': 83092758, 'licence': 'Data © Open...","501 00, Greece",NaN,Greece,NaN
5030,1970,1976-11-28,1856,"Moscow, Russia",Aeroflot,2415,Moscow - Leningrad,Tupolev TU-104B,CCCP-42471,021204,...,0.0,After taking off an artificial horizon failure...,73,73,Aeroflot,"{'place_id': 386169025, 'licence': 'Data © Ope...","Central Federal District, Russia",NaN,Russia,NaN
5031,1970,1976-12-12,2326,"Cape May, New Jersey",Atlantic City Airlines,977,Atlantic City - Cap May,de Havilland Canada DHC-6 Twin Otter 300,N101AC,262,...,0.0,The plane crashed short of the runway in rain....,10,3,Atlantic City Airlines,"{'place_id': 1881913, 'licence': 'Data © OpenS...","New Jersey, United States",NaN,United States,NaN
5032,1970,1976-12-17,1940,"Ust-Kut, Russia",Aeroflot,None,Ust-Kut - Kirensk,Yakovlev 40,CCCP-88208,9631349,...,0.0,The cargo plane crashed into trees on takeoff....,7,7,Aeroflot,"{'place_id': 224774650, 'licence': 'Data © Ope...","Siberian Federal District, Russia",NaN,Russia,NaN


In [ ]:
def get_code_2(x):
    try:
        return df_country_codes_updated.loc[x]["alpha-3"]
    except:
        return float("nan")

In [ ]:
df_country_codes_["Country_Geopy_"] = df_country_codes_.name_geopy.apply(unpack_locations_part)

In [ ]:
df_country_codes_

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,name_geopy,Country_Geopy_
Country_Geopy,,,,,,,,,,,,,
Afghanistan,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN,"{'place_id': 220100631, 'licence': 'Data © Ope...",Afghanistan
"Åland Islands, Finland",Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,None,150.0,154.0,NaN,"{'place_id': 180167715, 'licence': 'Data © Ope...",Finland
Albania,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN,"{'place_id': 83081284, 'licence': 'Data © Open...",Albania
Algeria,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 77146891, 'licence': 'Data © Open...",Algeria
"American Samoa, United States",American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,None,9.0,61.0,NaN,"{'place_id': 32805879, 'licence': 'Data © Open...",United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Wallis and Futuna, France",Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,None,9.0,61.0,NaN,"{'place_id': 33145748, 'licence': 'Data © Open...",France
"Dakhla-Oued Ed-Dahab, Morocco",Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 288513039, 'licence': 'Data © Ope...",Morocco
Yemen,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,None,142.0,145.0,NaN,"{'place_id': 69324426, 'licence': 'Data © Open...",Yemen


In [ ]:
df_country_codes_

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,name_geopy,Country_Geopy_
Country_Geopy,,,,,,,,,,,,,
Afghanistan,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN,"{'place_id': 220100631, 'licence': 'Data © Ope...",Afghanistan
"Åland Islands, Finland",Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,None,150.0,154.0,NaN,"{'place_id': 180167715, 'licence': 'Data © Ope...",Finland
Albania,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN,"{'place_id': 83081284, 'licence': 'Data © Open...",Albania
Algeria,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 77146891, 'licence': 'Data © Open...",Algeria
"American Samoa, United States",American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,None,9.0,61.0,NaN,"{'place_id': 32805879, 'licence': 'Data © Open...",United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Wallis and Futuna, France",Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,None,9.0,61.0,NaN,"{'place_id': 33145748, 'licence': 'Data © Open...",France
"Dakhla-Oued Ed-Dahab, Morocco",Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 288513039, 'licence': 'Data © Ope...",Morocco
Yemen,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,None,142.0,145.0,NaN,"{'place_id': 69324426, 'licence': 'Data © Open...",Yemen


In [ ]:
# import json
# import pandas as pd
# import plotly.express as px

# with open("data/geojson-counties-fips.json", "r") as f:
#     counties = json.load(f)

# import pandas as pd
# df = pd.read_csv("data/fips-unemp-16.csv",
#                    dtype={"fips": str})

# fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
#                            color_continuous_scale="viridis",
#                            range_color=(0, 12),
#                         #    scope="usa",
#                            labels={'unemp':'unemployment rate'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [ ]:
# import plotly.express as px

# # Sample data: a list of dictionaries
# # Each dictionary represents a point with a latitude, longitude, and some value.
# data = [
#     {"lat": 34.0522, "lon": -118.2437, "value": 10},  # Los Angeles
#     {"lat": 40.7128, "lon": -74.0060, "value": 20},  # New York
#     {"lat": 37.7749, "lon": -122.4194, "value": 30},  # San Francisco
# ]

# # Convert the list of dictionaries to a DataFrame
# import pandas as pd
# df = pd.DataFrame(data)

# # Plot using scatter_geo
# fig = px.scatter_geo(df,
#                      lat="lat",
#                      lon="lon",
#                      size="value",  # Adjust the size of each point based on the "value"
#                      )

# # Show the figure
# fig.show()


In [ ]:
# import plotly.express as px

# fig = px.choropleth(locations=["Northern Ireland, United Kingdom"], color=[1,2,3],)
# fig.show()

In [ ]:
# import plotly.express as px

# # Assuming 'df' is your DataFrame with 'lat' and 'lon' columns
# px.set_mapbox_access_token('your_mapbox_access_token_here')

# fig = px.density_mapbox(df, lat='lat', lon='lon', z='value', radius=10,
#                         center=dict(lat=0, lon=180), zoom=0,
#                         mapbox_style="stamen-terrain")

# fig.show()


In [ ]:
# import plotly.express as px
# import pandas as pd

# df = pd.read_csv("location_coordinate.csv")

# fig = px.scatter_geo(df,lat='lat',lon='long', hover_name="id")
# fig.update_layout(title = 'World map', title_x=0.5)
# fig.show()

In [ ]:
# df.Date = df.Date.astype(str)

In [ ]:
# df.to_json("data/silver/1920_2024_geopy.json")